In [6]:
pip install transformers torch accelerate diffusers accelerate

In [ ]:
pip install coqui_tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 100.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 109.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
pip install pydub moviepy Pillow

In [4]:
pip install gradio

In [ ]:
from transformers import pipeline

class StoryGenerator:
    def __init__(self, model_name="gpt2"): # You can try other models like "distilgpt2", "microsoft/DialoGPT-small" or larger ones if you have GPU
        self.generator = pipeline('text-generation', model=model_name)

    def generate_story_segment(self, prompt, max_length=100, num_return_sequences=1):
        """Generates a text segment based on a prompt."""
        generated_text = self.generator(
            prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=0.9, # Controls creativity (higher = more creative)
            top_k=50,        # Limits sampling to top_k tokens
            top_p=0.95,      # Filters out low probability tokens
            pad_token_id=self.generator.tokenizer.eos_token_id # For GPT-2
        )
        return generated_text[0]['generated_text']

    def generate_full_story(self, main_prompt, num_segments=3, segment_length=100):
        """Generates a multi-segment story."""
        full_story = []
        current_prompt = main_prompt
        for i in range(num_segments):
            print(f"Generating segment {i+1}...")
            segment = self.generate_story_segment(current_prompt, max_length=segment_length)
            full_story.append(segment)
            # Use the end of the current segment as a prompt for the next
            current_prompt = segment[-int(segment_length/4):] # Take last 1/4 of segment as context

        return " ".join(full_story)

# Example Usage:
# story_gen = StoryGenerator()
# story_text = story_gen.generate_full_story("A brave knight set out on a quest to find a magical artifact.", num_segments=3, segment_length=150)
# print("Generated Story:\n", story_text)

In [ ]:
from diffusers import DiffusionPipeline
import torch

class ImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"): # "stabilityai/stable-diffusion-xl-base-1.0" for SDXL (requires more VRAM)
        self.pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
        if torch.cuda.is_available():
            self.pipe.to("cuda") # Move model to GPU if available

    def generate_image(self, prompt, output_path="image.png"):
        """Generates an image from a text prompt."""
        print(f"Generating image for prompt: '{prompt}'")
        image = self.pipe(prompt).images[0]
        image.save(output_path)
        print(f"Image saved to {output_path}")
        return output_path

# Example Usage:
# img_gen = ImageGenerator()
# # You need to extract meaningful scene descriptions from your story text.
# # This is a crucial step! For simplicity, let's use a direct prompt here.
# scene_prompt = "A majestic dragon flying over a medieval castle at sunset, highly detailed, fantasy art"
# img_path = img_gen.generate_image(scene_prompt, "dragon_castle.png")

In [ ]:
from TTS.api import TTS

class AudioGenerator:
    def __init__(self, model_name="tts_models/en/ljspeech/tacotron2-DDC"):
        # You can explore other models by running `TTS().list_models()`
        # For multi-speaker: "tts_models/en/vctk/vits"
        # For faster generation: "tts_models/en/ljspeech/fast_pitch"
        self.tts = TTS(model_name=model_name, progress_bar=False, gpu=torch.cuda.is_available())

    def generate_audio(self, text, output_path="audio.wav"):
        """Converts text to speech."""
        print(f"Generating audio for text (first 50 chars): '{text[:50]}...'")
        self.tts.tts_to_file(text=text, file_path=output_path)
        print(f"Audio saved to {output_path}")
        return output_path

# Example Usage:
# audio_gen = AudioGenerator()
# audio_path = audio_gen.generate_audio("The brave knight rode through the enchanted forest.", "knight_audio.wav")

ModuleNotFoundError: No module named 'TTS'

In [ ]:
pip install coqui_tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 108.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 110.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 115.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from TTS.api import TTS

class AudioGenerator:
    def __init__(self, model_name="tts_models/en/ljspeech/tacotron2-DDC"):
        # You can explore other models by running `TTS().list_models()`
        # For multi-speaker: "tts_models/en/vctk/vits"
        # For faster generation: "tts_models/en/ljspeech/fast_pitch"
        self.tts = TTS(model_name=model_name, progress_bar=False, gpu=torch.cuda.is_available())

    def generate_audio(self, text, output_path="audio.wav"):
        """Converts text to speech."""
        print(f"Generating audio for text (first 50 chars): '{text[:50]}...'")
        self.tts.tts_to_file(text=text, file_path=output_path)
        print(f"Audio saved to {output_path}")
        return output_path

# Example Usage:
# audio_gen = AudioGenerator()
# audio_path = audio_gen.generate_audio("The brave knight rode through the enchanted forest.", "knight_audio.wav")

In [ ]:
from moviepy.editor import ImageClip, AudioFileClip, concatenate_audioclips, concatenate_videoclips
import os

class MultimediaAssembler:
    def __init__(self, image_dir="generated_images", audio_dir="generated_audio"):
        self.image_dir = image_dir
        self.audio_dir = audio_dir
        os.makedirs(image_dir, exist_ok=True)
        os.makedirs(audio_dir, exist_ok=True)

    def create_story_video(self, story_segments, image_paths, audio_paths, output_video_path="story.mp4"):
        """
        Combines images and audio into a video.
        Assumes image_paths and audio_paths are aligned with story_segments.
        """
        if len(image_paths) != len(audio_paths):
            print("Warning: Number of images and audio segments do not match. Video might be off-sync.")

        clips = []
        for i in range(len(story_segments)):
            image_path = image_paths[i] if i < len(image_paths) else None
            audio_path = audio_paths[i] if i < len(audio_paths) else None

            if audio_path and os.path.exists(audio_path):
                audio_clip = AudioFileClip(audio_path)
                duration = audio_clip.duration
            else:
                audio_clip = None
                duration = 5 # Default duration if no audio

            if image_path and os.path.exists(image_path):
                img_clip = ImageClip(image_path, duration=duration)
            else:
                # Create a black clip or use a placeholder image
                img_clip = ImageClip("placeholder.png", duration=duration) # You'd need a placeholder image

            if audio_clip:
                img_clip = img_clip.set_audio(audio_clip)
            clips.append(img_clip)

        if not clips:
            print("No clips to create video.")
            return

        final_clip = concatenate_videoclips(clips, method="compose")
        print(f"Rendering video to {output_video_path}...")
        final_clip.write_videofile(output_video_path, fps=24, codec="libx264", audio_codec="aac")
        print("Video creation complete.")
        return output_video_path

# Example Usage (within the main script):
# assembler = MultimediaAssembler()
# # You'd have lists of generated image_paths and audio_paths
# # story_segments = ["...", "...", "..."]
# # image_paths = ["image1.png", "image2.png", "image3.png"]
# # audio_paths = ["audio1.wav", "audio2.wav", "audio3.wav"]
# # assembler.create_story_video(story_segments, image_paths, audio_paths, "my_ai_story.mp4")

  if event.key is 'enter':



In [8]:
pip install transformers torch accelerate diffusers accelerate coqui_tts moviepy pydub Pillow gradio

In [ ]:
python AI_StoryTeller.py

SyntaxError: invalid syntax (<ipython-input-4-54e533c27aed>, line 1)

In [ ]:
run AI_StoryTeller.py

Exception: File `'AI_StoryTeller.py'` not found.

In [7]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,387 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://security.ubunt

In [5]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [6]:
!pip install moviepy

In [ ]:
# --- StoryGenerator Class ---
from transformers import pipeline
import torch

class StoryGenerator:
    def __init__(self, model_name="gpt2"): # You can try other models like "distilgpt2", "microsoft/DialoGPT-small" or larger ones if you have GPU
        self.generator = pipeline('text-generation', model=model_name)

    def generate_story_segment(self, prompt, max_length=100, num_return_sequences=1):
        """Generates a text segment based on a prompt."""
        generated_text = self.generator(
            prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=0.9, # Controls creativity (higher = more creative)
            top_k=50,        # Limits sampling to top_k tokens
            top_p=0.95,      # Filters out low probability tokens
            pad_token_id=self.generator.tokenizer.eos_token_id # For GPT-2
        )
        return generated_text[0]['generated_text']

    def generate_full_story(self, main_prompt, num_segments=3, segment_length=150):
        """Generates a multi-segment story."""
        full_story = []
        current_prompt = main_prompt
        for i in range(num_segments):
            print(f"Generating story segment {i+1}...")
            segment = self.generate_story_segment(current_prompt, max_length=segment_length)
            full_story.append(segment)
            current_prompt = segment[-int(segment_length/4):] # Use last part as context

        return full_story # Return as list of segments for easier processing

# --- ImageGenerator Class ---
from diffusers import DiffusionPipeline
import torch
import os

class ImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        # Ensure model is downloaded. First run might take time.
        # Use low_cpu_mem_usage=True for larger models on Colab to optimize memory
        self.pipe = DiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16,
                                                      low_cpu_mem_usage=True # Crucial for Colab's RAM
                                                     )
        if torch.cuda.is_available():
            self.pipe.to("cuda") # Move model to GPU

    def generate_image(self, prompt, output_path="image.png"):
        print(f"Generating image for prompt: '{prompt}'")
        image = self.pipe(prompt).images[0]
        image.save(output_path)
        print(f"Image saved to {output_path}")
        return output_path

# --- AudioGenerator Class ---
from TTS.api import TTS
import torch
import os

class AudioGenerator:
    def __init__(self, model_name="tts_models/en/ljspeech/tacotron2-DDC"):
        # You can explore other models by running `TTS().list_models()`
        # For multi-speaker: "tts_models/en/vctk/vits"
        # For faster generation: "tts_models/en/ljspeech/fast_pitch"
        self.tts = TTS(model_name=model_name, progress_bar=False, gpu=torch.cuda.is_available())

    def generate_audio(self, text, output_path="audio.wav"):
        print(f"Generating audio for text (first 50 chars): '{text[:50]}...'")
        self.tts.tts_to_file(text=text, file_path=output_path)
        print(f"Audio saved to {output_path}")
        return output_path

# --- MultimediaAssembler Class ---
from moviepy.editor import ImageClip, AudioFileClip, concatenate_audioclips, concatenate_videoclips
import os
from PIL import Image # Make sure PIL/Pillow is imported

class MultimediaAssembler:
    def __init__(self, image_dir="generated_images", audio_dir="generated_audio"):
        self.image_dir = image_dir
        self.audio_dir = audio_dir
        os.makedirs(image_dir, exist_ok=True)
        os.makedirs(audio_dir, exist_ok=True)
        # Create a simple placeholder image if it doesn't exist
        if not os.path.exists("placeholder.png"):
            img = Image.new('RGB', (1024, 576), color = 'black') # Standard video size
            img.save("placeholder.png")


    def create_story_video(self, story_segments, image_paths, audio_paths, output_video_path="story.mp4"):
        if len(image_paths) != len(audio_paths):
            print("Warning: Number of images and audio segments do not match. Video might be off-sync.")

        clips = []
        for i in range(max(len(story_segments), len(image_paths), len(audio_paths))):
            image_path = image_paths[i] if i < len(image_paths) and os.path.exists(image_paths[i]) else "placeholder.png"
            audio_path = audio_paths[i] if i < len(audio_paths) and os.path.exists(audio_paths[i]) else None

            if audio_path:
                audio_clip = AudioFileClip(audio_path)
                duration = audio_clip.duration
            else:
                audio_clip = None
                duration = 5 # Default duration if no audio

            img_clip = ImageClip(image_path, duration=duration)

            if audio_clip:
                img_clip = img_clip.set_audio(audio_clip)
            clips.append(img_clip)

        if not clips:
            print("No clips to create video.")
            return

        final_clip = concatenate_videoclips(clips, method="compose")
        print(f"Rendering video to {output_video_path}...")
        final_clip.write_videofile(output_video_path, fps=24, codec="libx264", audio_codec="aac")
        print("Video creation complete.")
        return output_video_path

# --- Main Application Logic ---
def run_ai_storyteller(prompt):
    story_gen = StoryGenerator()
    img_gen = ImageGenerator()
    audio_gen = AudioGenerator()
    assembler = MultimediaAssembler()

    # 1. Generate Story
    story_segments = story_gen.generate_full_story(prompt, num_segments=3, segment_length=100)

    # 2. Generate Images and Audio for each segment
    image_paths = []
    audio_paths = []
    for i, segment in enumerate(story_segments):
        # For simplicity, using the first 50 chars of the segment as image prompt.
        # In a real application, you'd want more sophisticated scene extraction.
        image_prompt = segment.split('.')[0].strip() # Take first sentence as prompt
        if not image_prompt:
            image_prompt = "A generic fantasy scene" # Fallback

        img_path = os.path.join(assembler.image_dir, f"scene_{i+1}.png")
        audio_path = os.path.join(assembler.audio_dir, f"scene_{i+1}.wav")

        img_gen.generate_image(image_prompt, img_path)
        audio_gen.generate_audio(segment, audio_path)

        image_paths.append(img_path)
        audio_paths.append(audio_path)

    # 3. Assemble into video
    output_video_file = "generated_story.mp4"
    assembler.create_story_video(story_segments, image_paths, audio_paths, output_video_file)
    print(f"AI Storyteller finished. Video saved to {output_video_file}")
    return output_video_file # Return the path for Gradio to display

# --- Simple Gradio UI (Optional but Recommended for easy interaction) ---
import gradio as gr

def gradio_interface(prompt_text):
    output_path = run_ai_storyteller(prompt_text)
    # Display the video directly in the Gradio interface
    return output_path

if __name__ == '__main__':
    print("Starting AI Storyteller...")
    # Example usage (uncomment if you want to run directly without Gradio UI)
    # run_ai_storyteller("A young wizard discovers a hidden ancient scroll in a forgotten library.")

    iface = gr.Interface(
        fn=gradio_interface,
        inputs=gr.Textbox(lines=2, placeholder="Enter your story idea here... (e.g., A brave knight sets out on a quest)"),
        outputs=gr.Video(label="Generated Story Video"), # Change output to Video component
        title="AI-Powered Storyteller with Visuals and Audio",
        description="Enter a prompt, and the AI will generate a story, create images, and narrate it into a video!"
    )
    iface.launch(share=True) # share=True generates a public URL, useful for Colab

ModuleNotFoundError: No module named 'TTS'

In [4]:
pip install coqui_tts

In [ ]:
# --- StoryGenerator Class ---
from transformers import pipeline
import torch

class StoryGenerator:
    def __init__(self, model_name="gpt2"): # You can try other models like "distilgpt2", "microsoft/DialoGPT-small" or larger ones if you have GPU
        self.generator = pipeline('text-generation', model=model_name)

    def generate_story_segment(self, prompt, max_length=100, num_return_sequences=1):
        """Generates a text segment based on a prompt."""
        generated_text = self.generator(
            prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=0.9, # Controls creativity (higher = more creative)
            top_k=50,        # Limits sampling to top_k tokens
            top_p=0.95,      # Filters out low probability tokens
            pad_token_id=self.generator.tokenizer.eos_token_id # For GPT-2
        )
        return generated_text[0]['generated_text']

    def generate_full_story(self, main_prompt, num_segments=3, segment_length=150):
        """Generates a multi-segment story."""
        full_story = []
        current_prompt = main_prompt
        for i in range(num_segments):
            print(f"Generating story segment {i+1}...")
            segment = self.generate_story_segment(current_prompt, max_length=segment_length)
            full_story.append(segment)
            current_prompt = segment[-int(segment_length/4):] # Use last part as context

        return full_story # Return as list of segments for easier processing

# --- ImageGenerator Class ---
from diffusers import DiffusionPipeline
import torch
import os

class ImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        # Ensure model is downloaded. First run might take time.
        # Use low_cpu_mem_usage=True for larger models on Colab to optimize memory
        self.pipe = DiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16,
                                                      low_cpu_mem_usage=True # Crucial for Colab's RAM
                                                     )
        if torch.cuda.is_available():
            self.pipe.to("cuda") # Move model to GPU

    def generate_image(self, prompt, output_path="image.png"):
        print(f"Generating image for prompt: '{prompt}'")
        image = self.pipe(prompt).images[0]
        image.save(output_path)
        print(f"Image saved to {output_path}")
        return output_path

# --- AudioGenerator Class ---
from TTS.api import TTS
import torch
import os

class AudioGenerator:
    def __init__(self, model_name="tts_models/en/ljspeech/tacotron2-DDC"):
        # You can explore other models by running `TTS().list_models()`
        # For multi-speaker: "tts_models/en/vctk/vits"
        # For faster generation: "tts_models/en/ljspeech/fast_pitch"
        self.tts = TTS(model_name=model_name, progress_bar=False, gpu=torch.cuda.is_available())

    def generate_audio(self, text, output_path="audio.wav"):
        print(f"Generating audio for text (first 50 chars): '{text[:50]}...'")
        self.tts.tts_to_file(text=text, file_path=output_path)
        print(f"Audio saved to {output_path}")
        return output_path

# --- MultimediaAssembler Class ---
from moviepy.editor import ImageClip, AudioFileClip, concatenate_audioclips, concatenate_videoclips
import os
from PIL import Image # Make sure PIL/Pillow is imported

class MultimediaAssembler:
    def __init__(self, image_dir="generated_images", audio_dir="generated_audio"):
        self.image_dir = image_dir
        self.audio_dir = audio_dir
        os.makedirs(image_dir, exist_ok=True)
        os.makedirs(audio_dir, exist_ok=True)
        # Create a simple placeholder image if it doesn't exist
        if not os.path.exists("placeholder.png"):
            img = Image.new('RGB', (1024, 576), color = 'black') # Standard video size
            img.save("placeholder.png")


    def create_story_video(self, story_segments, image_paths, audio_paths, output_video_path="story.mp4"):
        if len(image_paths) != len(audio_paths):
            print("Warning: Number of images and audio segments do not match. Video might be off-sync.")

        clips = []
        for i in range(max(len(story_segments), len(image_paths), len(audio_paths))):
            image_path = image_paths[i] if i < len(image_paths) and os.path.exists(image_paths[i]) else "placeholder.png"
            audio_path = audio_paths[i] if i < len(audio_paths) and os.path.exists(audio_paths[i]) else None

            if audio_path:
                audio_clip = AudioFileClip(audio_path)
                duration = audio_clip.duration
            else:
                audio_clip = None
                duration = 5 # Default duration if no audio

            img_clip = ImageClip(image_path, duration=duration)

            if audio_clip:
                img_clip = img_clip.set_audio(audio_clip)
            clips.append(img_clip)

        if not clips:
            print("No clips to create video.")
            return

        final_clip = concatenate_videoclips(clips, method="compose")
        print(f"Rendering video to {output_video_path}...")
        final_clip.write_videofile(output_video_path, fps=24, codec="libx264", audio_codec="aac")
        print("Video creation complete.")
        return output_video_path

# --- Main Application Logic ---
def run_ai_storyteller(prompt):
    story_gen = StoryGenerator()
    img_gen = ImageGenerator()
    audio_gen = AudioGenerator()
    assembler = MultimediaAssembler()

    # 1. Generate Story
    story_segments = story_gen.generate_full_story(prompt, num_segments=3, segment_length=100)

    # 2. Generate Images and Audio for each segment
    image_paths = []
    audio_paths = []
    for i, segment in enumerate(story_segments):
        # For simplicity, using the first 50 chars of the segment as image prompt.
        # In a real application, you'd want more sophisticated scene extraction.
        image_prompt = segment.split('.')[0].strip() # Take first sentence as prompt
        if not image_prompt:
            image_prompt = "A generic fantasy scene" # Fallback

        img_path = os.path.join(assembler.image_dir, f"scene_{i+1}.png")
        audio_path = os.path.join(assembler.audio_dir, f"scene_{i+1}.wav")

        img_gen.generate_image(image_prompt, img_path)
        audio_gen.generate_audio(segment, audio_path)

        image_paths.append(img_path)
        audio_paths.append(audio_path)

    # 3. Assemble into video
    output_video_file = "generated_story.mp4"
    assembler.create_story_video(story_segments, image_paths, audio_paths, output_video_file)
    print(f"AI Storyteller finished. Video saved to {output_video_file}")
    return output_video_file # Return the path for Gradio to display

# --- Simple Gradio UI (Optional but Recommended for easy interaction) ---
import gradio as gr

def gradio_interface(prompt_text):
    output_path = run_ai_storyteller(prompt_text)
    # Display the video directly in the Gradio interface
    return output_path

if __name__ == '__main__':
    print("Starting AI Storyteller...")
    # Example usage (uncomment if you want to run directly without Gradio UI)
    # run_ai_storyteller("A young wizard discovers a hidden ancient scroll in a forgotten library.")

    iface = gr.Interface(
        fn=gradio_interface,
        inputs=gr.Textbox(lines=2, placeholder="Enter your story idea here... (e.g., A brave knight sets out on a quest)"),
        outputs=gr.Video(label="Generated Story Video"), # Change output to Video component
        title="AI-Powered Storyteller with Visuals and Audio",
        description="Enter a prompt, and the AI will generate a story, create images, and narrate it into a video!"
    )
    iface.launch(share=True) # share=True generates a public URL, useful for Colab

  if event.key is 'enter':



ModuleNotFoundError: No module named 'gradio'

In [2]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


In [11]:
# Standard library imports (already covered in previous cells but good practice to list)
import os
import shutil # For clearing directories
import torch # For GPU checks and model loading
from PIL import Image # For placeholder image

# Third-party library imports
from transformers import pipeline
from diffusers import DiffusionPipeline
from TTS.api import TTS
from moviepy.editor import ImageClip, AudioFileClip, concatenate_audioclips, concatenate_videoclips
import gradio as gr


# --- StoryGenerator Class ---
class StoryGenerator:
    def __init__(self, model_name="gpt2"): # Default to gpt2 for compatibility, consider gpt2-medium for better quality
        print(f"Initializing StoryGenerator with model: {model_name}")
        self.generator = pipeline('text-generation', model=model_name)
        # Store tokenizer max length for context management
        self.max_model_input_length = self.generator.tokenizer.model_max_length or 1024 # Default if not found

    def generate_story_segment(self, prompt, max_length=100, num_return_sequences=1):
        """Generates a text segment based on a prompt."""
        generated_text_output = self.generator(
            prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=0.8, # Slightly lowered temperature for more coherence
            top_k=50,
            top_p=0.95,
            pad_token_id=self.generator.tokenizer.eos_token_id,
            truncation=True # Ensure input prompt is truncated if it exceeds model max_length internally
        )

        # The generated text might repeat the prompt. We only want the new part.
        generated_segment = generated_text_output[0]['generated_text']

        # Heuristic to remove the prompt from the beginning of the generated text
        # This is important for clean accumulation of story segments
        if generated_segment.startswith(prompt.strip()):
            generated_segment = generated_segment[len(prompt.strip()):].strip()
            # If the stripping leaves it empty or too short, revert or take a default
            if len(generated_segment) < 20 and len(generated_text_output[0]['generated_text']) > len(prompt.strip()):
                generated_segment = generated_text_output[0]['generated_text'] # Fallback

        # Ensure the segment ends reasonably (e.g., at a sentence boundary if possible)
        # This can help improve how it flows into the next segment
        if '.' in generated_segment:
            generated_segment = generated_segment.rsplit('.', 1)[0] + '.'
        elif '?' in generated_segment:
            generated_segment = generated_segment.rsplit('?', 1)[0] + '?'
        elif '!' in generated_segment:
            generated_segment = generated_segment.rsplit('!', 1)[0] + '!'

        return generated_segment.strip()


    def generate_full_story(self, main_prompt, num_segments=3, segment_length=150):
        full_story_list = [] # Stores individual segments
        current_context = main_prompt # This will accumulate the story context fed to the model

        # Adjust segment_length if it's too large for the model's context window
        # This ensures there's always space for the prompt and remaining context
        if segment_length >= self.max_model_input_length - 100: # Leave at least 100 tokens for prompt/overhead
            segment_length = self.max_model_input_length // 2 # A safer default

        for i in range(num_segments):
            print(f"Generating story segment {i+1}...")

            # Prepare the prompt for the current segment generation
            # It should guide the model to continue the story
            # Add a small instruction to encourage continuation for better coherence
            segment_input_prompt = f"{current_context.strip()}\n\n" \
                                   f"Continue this story naturally, expanding on the last events: "

            # Calculate the maximum tokens available for the actual story context,
            # considering the length of the instruction and the new segment length
            instruction_tokens = len(self.generator.tokenizer.encode("Continue this story naturally, expanding on the last events: "))
            max_context_tokens = self.max_model_input_length - instruction_tokens - segment_length

            encoded_input_context = self.generator.tokenizer.encode(current_context)

            if len(encoded_input_context) > max_context_tokens:
                # Trim the context from the beginning if it's too long
                # Decode only the relevant part to avoid re-tokenizing the whole thing
                current_context_truncated = self.generator.tokenizer.decode(encoded_input_context[-max_context_tokens:], skip_special_tokens=True)
                segment_input_prompt = f"{current_context_truncated.strip()}\n\n" \
                                       f"Continue this story naturally, expanding on the last events: "
            # Else, use the full current_context as segment_input_prompt already has it

            segment = self.generate_story_segment(segment_input_prompt, max_length=segment_length)

            full_story_list.append(segment)
            # Accumulate the newly generated segment to the context for the next iteration
            current_context += " " + segment

        return full_story_list # Return as a list of segments


# --- ImageGenerator Class ---
class ImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        print(f"Initializing ImageGenerator with model: {model_id}")
        # Ensure model is downloaded. First run might take time.
        # Use low_cpu_mem_usage=True for larger models on Colab to optimize memory
        self.pipe = DiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16,
                                                      low_cpu_mem_usage=True # Crucial for Colab's RAM
                                                     )
        if torch.cuda.is_available():
            self.pipe.to("cuda") # Move model to GPU

    def generate_image(self, prompt, output_path="image.png"):
        print(f"Generating image for prompt: '{prompt}'")
        # Added a negative prompt to improve image quality
        negative_prompt = "blurry, low resolution, bad quality, distorted, ugly, tiling, poorly drawn hands, poor anatomy, bad composition"
        image = self.pipe(prompt, negative_prompt=negative_prompt).images[0]
        image.save(output_path)
        print(f"Image saved to {output_path}")
        return output_path

# --- AudioGenerator Class ---
class AudioGenerator:
    def __init__(self, model_name="tts_models/en/ljspeech/tacotron2-DDC"):
        print(f"Initializing AudioGenerator with model: {model_name}")
        self.tts = TTS(model_name=model_name, progress_bar=False, gpu=torch.cuda.is_available())

    def generate_audio(self, text, output_path="audio.wav"):
        print(f"Generating audio for text (first 50 chars): '{text[:50]}...'")
        self.tts.tts_to_file(text=text, file_path=output_path)
        print(f"Audio saved to {output_path}")
        return output_path

# --- MultimediaAssembler Class ---
class MultimediaAssembler:
    def __init__(self, image_dir="generated_images", audio_dir="generated_audio"):
        print(f"Initializing MultimediaAssembler...")
        self.image_dir = image_dir
        self.audio_dir = audio_dir
        os.makedirs(image_dir, exist_ok=True)
        os.makedirs(audio_dir, exist_ok=True)
        # Create a simple placeholder image if it doesn't exist
        if not os.path.exists("placeholder.png"):
            img = Image.new('RGB', (1024, 576), color = 'black') # Standard video size
            img.save("placeholder.png")


    def create_story_video(self, story_segments, image_paths, audio_paths, output_video_path="story.mp4"):
        print("Assembling video...")
        if len(image_paths) != len(audio_paths):
            print("Warning: Number of images and audio segments do not match. Video might be off-sync.")

        clips = []
        # Iterate based on the longest list, using placeholders for missing elements
        max_len = max(len(story_segments), len(image_paths), len(audio_paths))

        for i in range(max_len):
            image_path = image_paths[i] if i < len(image_paths) and os.path.exists(image_paths[i]) else "placeholder.png"
            audio_path = audio_paths[i] if i < len(audio_paths) and os.path.exists(audio_paths[i]) else None

            audio_clip = None
            duration = 5 # Default duration if no audio or audio clip is too short

            if audio_path:
                try:
                    audio_clip = AudioFileClip(audio_path)
                    duration = audio_clip.duration
                except Exception as e:
                    print(f"Warning: Could not load audio clip {audio_path}: {e}. Using default duration.")
                    audio_clip = None # Fallback to no audio if error

            # Ensure image clip is created with the determined duration
            try:
                img_clip = ImageClip(image_path, duration=duration)
            except Exception as e:
                print(f"Warning: Could not load image clip {image_path}: {e}. Using black placeholder.")
                img_clip = ImageClip("placeholder.png", duration=duration)


            if audio_clip:
                img_clip = img_clip.set_audio(audio_clip)
            clips.append(img_clip)

        if not clips:
            print("No clips to create video.")
            return

        final_clip = concatenate_videoclips(clips, method="compose")
        print(f"Rendering video to {output_video_path}...")
        # Use a simpler preset for faster rendering on Colab
        final_clip.write_videofile(output_video_path, fps=24, codec="libx264", audio_codec="aac", preset='medium')
        print("Video creation complete.")
        return output_video_path

# --- Main Application Logic ---
def run_ai_storyteller(prompt):
    # Instantiate classes
    story_gen = StoryGenerator(model_name="gpt2") # Consider "gpt2-medium" if GPU allows for better quality
    img_gen = ImageGenerator(model_id="runwayml/stable-diffusion-v1-5") # Consider "stabilityai/stable-diffusion-xl-base-1.0" for better images (more VRAM)
    audio_gen = AudioGenerator(model_name="tts_models/en/ljspeech/tacotron2-DDC") # Default, explore others if needed
    assembler = MultimediaAssembler()

    print(f"\n--- Starting story generation for prompt: '{prompt}' ---")
    # 1. Generate Story
    # Increased segments, reduced segment length for better manageability
    story_segments = story_gen.generate_full_story(prompt, num_segments=4, segment_length=120)

    # 2. Generate Images and Audio for each segment
    image_paths = []
    audio_paths = []
    for i, segment in enumerate(story_segments):
        print(f"\n--- Processing segment {i+1}/{len(story_segments)} ---")
        # For simplicity, using the first sentence of the segment as image prompt.
        # More advanced: use a small LLM to extract key visual elements.
        image_prompt_candidate = segment.split('.')[0].strip() # Take first sentence as prompt
        if not image_prompt_candidate:
            image_prompt = prompt # Fallback to original prompt if no sentence
        else:
            image_prompt = image_prompt_candidate # Use the first sentence

        # Add some descriptive tags for better image generation
        # These are crucial for good image quality!
        image_prompt += ", highly detailed, cinematic lighting, dramatic, realistic, 8k"


        img_path = os.path.join(assembler.image_dir, f"scene_{i+1}.png")
        audio_path = os.path.join(assembler.audio_dir, f"scene_{i+1}.wav")

        try:
            img_gen.generate_image(image_prompt, img_path)
        except Exception as e:
            print(f"ERROR: Image generation failed for segment {i+1}: {e}. Using placeholder.")
            # Fallback to placeholder image if generation fails
            Image.new('RGB', (1024, 576), color = 'black').save(img_path)

        try:
            audio_gen.generate_audio(segment, audio_path)
        except Exception as e:
            print(f"ERROR: Audio generation failed for segment {i+1}: {e}. Skipping audio for this segment.")
            # Ensure audio file exists for moviepy, even if empty/short
            with open(audio_path, 'w') as f:
                f.write('') # Create an empty file if audio generation fails


        image_paths.append(img_path)
        audio_paths.append(audio_path)

    print(f"\n--- All segments processed. Assembling video ---")
    # 3. Assemble into video
    output_video_file = "generated_story.mp4"
    assembler.create_story_video(story_segments, image_paths, audio_paths, output_video_file)
    print(f"\nAI Storyteller finished. Video saved to {output_video_file}")
    return output_video_file # Return the path for Gradio to display

# --- Gradio UI ---
class StreamlitHack:
    """A dummy class to make the print statements look like a Streamlit log."""
    def write(self, text):
        print(text)
st = StreamlitHack() # Initialize this for potential future Streamlit-like logging

def gradio_interface(prompt_text):
    if not prompt_text:
        return None # Gradio will display empty or no output

    # Clear previous output directories to ensure fresh start
    print("\n--- Clearing previous outputs ---")
    if os.path.exists("generated_story.mp4"):
        os.remove("generated_story.mp4")
    if os.path.exists("generated_images"):
        shutil.rmtree("generated_images")
    if os.path.exists("generated_audio"):
        shutil.rmtree("generated_audio")

    # Recreate directories (Assembler also does this, but good to be explicit for clearing)
    os.makedirs("generated_images", exist_ok=True)
    os.makedirs("generated_audio", exist_ok=True)

    try:
        output_path = run_ai_storyteller(prompt_text)
        return output_path
    except Exception as e:
        print(f"\n--- FATAL ERROR during story generation ---")
        print(f"Error details: {e}")
        import traceback
        traceback.print_exc() # Print full traceback to Colab output
        gr.Warning(f"An error occurred: {e}. Check Colab output for details.")
        return None # Indicate failure to Gradio

if __name__ == '__main__':
    print("Starting AI Storyteller...")
    iface = gr.Interface(
        fn=gradio_interface,
        inputs=gr.Textbox(lines=2, placeholder="Enter your story idea here... (e.g., A brave knight sets out on a quest)"),
        outputs=gr.Video(label="Generated Story Video", value=None), # Set initial value to None
        title="AI-Powered Storyteller with Visuals and Audio",
        description="Enter a prompt, and the AI will generate a story, create images, and narrate it into a video! "
                    "Note: First run downloads large AI models and may take a while. "
                    "Monitor the Colab output for detailed progress and errors."
    )
    iface.launch(share=True) # share=True generates a public URL, useful for Colab

Starting AI Storyteller...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dea285b13f1d679b2a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
